In [2]:
import cv2
import numpy as np
import scipy.stats as st
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
path = "CVFinalPics/IMG_20180130_174159662.jpg"

In [4]:
#Read a grayscale image
def read_image(path):
    image = cv2.imread(path,0)
    return image

In [ ]:
#Resize the input image
def resize_image(image,x,y):
    resized_image = cv2.resize(image,(x,y))
    return resized_image

In [5]:
#Displays an image
def display_image(image):
    plt.imshow(image)
    plt.show()

In [ ]:
#Gaussian blurring for noise removal
def gaussian_smoothing(image,filter_x,filter_y):
    blur_image = gaussian_blur(image,filter_x,filter_y)
    return blur_image

In [6]:
#Returns a gaussian filter with the specified kernel size and sigma
def gaussian_kernel(kernel_size,sigma):
    interval = (2*sigma+1.)/(kernel_size)
    x = np.linspace(-sigma-interval/2., sigma+interval/2., kernel_size+1)
    kern1d = np.diff(st.norm.cdf(x))
    kernel_raw = np.sqrt(np.outer(kern1d, kern1d))
    kernel = kernel_raw/kernel_raw.sum()
    return kernel

In [7]:
def harris_corner_detector(image,kernel,sobel_filter_size,hc_constant,threshold):
    partial_der_x = cv2.Sobel(image,cv2.CV_64F,1,0,sobel_filter_size)
    partial_der_y = cv2.Sobel(image,cv2.CV_64F,0,1,sobel_filter_size)
    prod_der_xx = np.multiply(partial_der_x,partial_der_x)
    prod_der_yy = np.multiply(partial_der_y,partial_der_y)
    prod_der_xy = np.multiply(partial_der_x,partial_der_y)
    
    sum_xx = cv2.filter2D(prod_der_xx, -1, kernel)
    sum_yy = cv2.filter2D(prod_der_yy, -1, kernel)
    sum_xy = cv2.filter2D(prod_der_xy, -1, kernel)
    
    trace =  sum_xx + sum_yy
    det = (sum_xx * sum_yy)-(sum_xy**2)
    response = det - hc_constant*(trace**2)
    
    result_image = image.copy()
    result_image = cv2.cvtColor(result_image,cv2.COLOR_GRAY2RGB)
    
    corner_points = []
    rows,cols = response.shape
    for i in range(rows):
        for j in range(cols):
            if(response[i,j]>threshold*response.max()):
                corner_points.append([i,j,response[i,j]])
                result_image.itemset((i,j,0),255)
                result_image.itemset((i,j,1),0)
                result_image.itemset((i,j,2),0)

    return result_image,corner_points

In [51]:
def projection_transform_matrix(A,B):
    C = np.zeros((8,9))
    ones = np.ones(3)
    
    P = A[0:3]    
    P = np.c_[P,ones]
    
    Q = A[3]
    Q = np.append(Q,1)
    temp1 = np.linalg.lstsq(P, Q)[0]
    temp1= np.repeat(temp1,3).reshape(3,3).T    
    scaled_P = np.multiply(P,temp1)
    
    R = B[0:3]
    R = np.c_[R,ones]
    
    S = B[3]
    S = np.append(S,1)
    temp2 = np.linalg.lstsq(R, S)[0]
    temp2 = np.repeat(temp2,3).reshape(3,3).T
    scaled_Q = np.multiply(Q,temp2)
    
    scaled_P_inv = np.linalg.inv(scaled_P)
    H = np.dot(scaled_P_inv,scaled_Q)
    return H

In [12]:
cpoints1 = np.array([[187, 285],[238, 475],[647, 269],[566, 523]])
A = cpoints1[0:3]

In [4]:
A

array([[187, 285],
       [238, 475],
       [647, 269]])

In [10]:
ones = np.ones(A.shape[0])
A = np.c_[A,ones]

In [11]:
A

array([[ 187.,  285.,    1.],
       [ 238.,  475.,    1.],
       [ 647.,  269.,    1.]])

In [14]:
Q = cpoints1[3]

In [16]:
Q
Q = np.append(Q,1)

In [17]:
Q

array([566, 523,   1])

In [39]:
A = cpoints1
ones = np.ones(3)
P = A[0:3]    
P = np.c_[P,ones]    
Q = A[3]
Q = np.append(Q,1)
x = np.linalg.lstsq(P, Q)[0]

LinAlgError: Incompatible dimensions

In [37]:
P

array([[ 187.,  285.,    1.,    1.],
       [ 238.,  475.,    1.,    1.],
       [ 647.,  269.,    1.,    1.]])

In [38]:
Q

array([566, 523,   1,   1])

In [19]:
x

array([ -1.22469847e+00,   1.02419062e-01,   7.65829181e+02])

In [30]:
y = np.repeat(x,3)

In [31]:
y

array([ -1.22469847e+00,  -1.22469847e+00,  -1.22469847e+00,
         1.02419062e-01,   1.02419062e-01,   1.02419062e-01,
         7.65829181e+02,   7.65829181e+02,   7.65829181e+02])

In [32]:
y.reshape(3,3).T


array([[ -1.22469847e+00,   1.02419062e-01,   7.65829181e+02],
       [ -1.22469847e+00,   1.02419062e-01,   7.65829181e+02],
       [ -1.22469847e+00,   1.02419062e-01,   7.65829181e+02]])

In [52]:
cpoints1 = np.array([[187, 285],[238, 475],[647, 269],[566, 523]])
cpoints2 = np.array([[161,109],[239, 300],[639, 55],[560, 317]])

In [53]:
H = projection_transform_matrix(cpoints1,cpoints2)

In [54]:
H

array([[ -1.21476955e-16,   5.59180434e-17,   3.05384355e-17],
       [  1.45061550e-15,  -1.50081235e-15,   6.93382775e-16],
       [ -9.27731405e-01,  -5.18764588e-01,   1.10324618e+00]])

In [ ]:
all_corner_points1 = np.array([[120,249],[140,267],[165,202],[165,328],[310,195],[255,325],[337,319],[426,327],[522,322],[625,324],[568,433],[565,499],[565,520],[541,520],[451,516],[390,511],[154,513],[114,340],[462,394],[186,405]])

In [ ]:
all_corner_points1 = np.array([[85,75],[105,97],[124,18],[145,153],[171,138],[235,145],[321,138],[412,132],[508,124],[616,115],[561,235],[559,295],[558,315],[534,315],[447,318],[390,321],[163,337],[94,169],[391,274],[177,237]])